In [1]:
import os
import glob
import numpy as np 
import xarray as xr
import pandas as pd
import datetime
from datetime import date, timedelta
import dask

# Plotting utils 
import matplotlib
import matplotlib.pyplot as plt 
import cartopy
import cartopy.crs as ccrs

import warnings
warnings.filterwarnings("ignore")


In [2]:
import dask
dask.config.set({'logging.distributed': 'error'})

from dask_jobqueue import PBSCluster

# For Casper
cluster = PBSCluster(
    queue="casper",
    walltime="02:00:00",
    account="P93300041",
    memory="4GB",
    resource_spec="select=1:ncpus=1:mem=4GB",
    cores=1,
    processes=1,
)

from dask.distributed import Client

# Connect client to the remote dask workers
client = Client(cluster)
client

cluster.scale(8)
client.wait_for_workers(8)


In [3]:
client

<Client: 'tcp://128.117.208.109:35483' processes=5 threads=5, memory=18.65 GiB>

In [3]:
# print(cluster.job_script())

In [13]:
# # Scheduler address from the launch_cluster notebook.
# from dask.distributed import Client

# client = Client("tcp://128.117.208.109:42179")

In [4]:
def preprocess_r_files(DS): 
    keepVars = ['T_VEG','T_REF2M','T_SOISNO','rh10','PREC10_VALUE']

    DS = DS[keepVars]

    return DS

## Look at restart files: S2S

In [40]:
s2s_dir = '/glade/campaign/cesm/development/cross-wg/S2S/land/rest/'

listFiles_h0_1990s = np.sort(glob.glob(s2s_dir+'199?*/*.clm2.h0.*.nc'))
listFiles_r_1990s  = np.sort(glob.glob(s2s_dir+'199?-??-01*/*.clm2.r.*.nc')) # Select only the first of the month to match SMYLE

listFiles_h0_2000s = np.sort(glob.glob(s2s_dir+'200?*/*.clm2.h0.*.nc'))
listFiles_r_2000s  = np.sort(glob.glob(s2s_dir+'200?-??-01*/*.clm2.r.*.nc'))

# listFiles_h0_2010s = np.sort(glob.glob(s2s_dir+'201?*/*.clm2.h0.*.nc'))
# listFiles_r_2010s  = np.sort(glob.glob(s2s_dir+'201?*/*.clm2.r.*.nc'))


In [56]:
%%time

## Append all into one list
listFiles_h0 = np.append(listFiles_h0_1990s, listFiles_h0_2000s)
listFiles_r  = np.append(listFiles_r_1990s,  listFiles_r_2000s)

# listFiles_h0 = np.append(listFiles_h0, listFiles_h0_2010s)
# listFiles_r  = np.append(listFiles_r,  listFiles_r_2010s)

len(listFiles_r)

CPU times: user 1.91 ms, sys: 0 ns, total: 1.91 ms
Wall time: 1.92 ms


323

In [85]:
## Get unique initial condition files  
for iFile in range(len(listFiles_h0)):
    fileString = listFiles_h0[iFile].split('/')[-1].split('.')[-2][0:7]
    
    if iFile==0:
        uniqFile = fileString
        paths    = listFiles_h0[iFile]
    else: 
        if fileString not in uniqFile:
            uniqFile = np.append(uniqFile, fileString)
            paths    = np.append(paths, listFiles_h0[iFile])



In [86]:
print(len(paths),'\n',len(listFiles_h0))

165 
 9822


In [111]:
%%time 

## Open h0 files 
S2S_h0 = xr.open_mfdataset(paths, concat_dim='time', 
                           combine='nested', decode_times=True, 
                           chunks = {"time": -1, "lon": -1, "lat": -1},
                           data_vars='minimal', parallel=True)


CPU times: user 41.8 s, sys: 2.87 s, total: 44.7 s
Wall time: 1min 48s


In [71]:
## Get unique restart files  
for iFile in range(len(listFiles_r)):
    fileString = listFiles_r[0].split('/')[-1].split('.')[-2]
    
    if iFile==0:
        uniqFile_r = fileString
        paths_r    = listFiles_r[iFile]
    else: 
        if fileString not in uniqFile_r:
            uniqFile_r = np.append(uniqFile, fileString)
            paths_r    = np.append(paths_r, listFiles_r[iFile])



In [72]:
fileString

'1990-01-01-00000'

In [73]:
print(len(paths_r),'\n',len(listFiles_r))

124 
 323


In [74]:
%%time 

## Open h0 files 
S2S_r = xr.open_mfdataset(paths_r, preprocess=preprocess_r_files, concat_dim='time', 
                           combine='nested', decode_times=True, 
                           chunks = {"time": -1, "lon": -1, "lat": -1},
                           data_vars='minimal', 
                         #  compat="override"
                         )


CPU times: user 266 ms, sys: 47.8 ms, total: 314 ms
Wall time: 371 ms


## Look at restart files: SMYLE

History files are monthly

In [26]:
%%time

smyle_dir = '/glade/campaign/cesm/development/espwg/SMYLE/initial_conditions/CLM5_SMYLE-Trendy/'

listFiles_h0_1990s = np.sort(glob.glob(smyle_dir+'hist/*.clm2.h0.199*.nc'))
listFiles_r_1990s  = np.sort(glob.glob(smyle_dir+'rest/199?*/*.clm2.r.*.nc'))

listFiles_h0_2000s = np.sort(glob.glob(smyle_dir+'hist/*.clm2.h0.200*.nc'))
listFiles_r_2000s  = np.sort(glob.glob(smyle_dir+'rest/200?*/*.clm2.r.*.nc'))

## Append all into one list
listFiles_h0 = np.append(listFiles_h0_1990s, listFiles_h0_2000s)
listFiles_r  = np.append(listFiles_r_1990s,  listFiles_r_2000s)


CPU times: user 38 ms, sys: 16.4 ms, total: 54.4 ms
Wall time: 337 ms


In [27]:
## Get unique initial condition files  
for iFile in range(len(listFiles_h0)):
    fileString = listFiles_h0[iFile].split('/')[-1]
    
    if iFile==0:
        uniqFile = fileString
        paths    = listFiles_h0[iFile]
    else: 
        if fileString not in uniqFile:
            uniqFile = np.append(uniqFile, fileString)
            paths    = np.append(paths, listFiles_h0[iFile])



In [28]:
%%time 

## Open h0 files 
smyle_h0 = xr.open_mfdataset(paths, concat_dim='time', 
                           combine='nested', decode_times=True, 
                           chunks = {"time": -1, "lon": -1, "lat": -1},
                           data_vars='minimal', parallel=True)


CPU times: user 1min 43s, sys: 2.84 s, total: 1min 46s
Wall time: 2min 55s


In [29]:
## Get unique restart files  
for iFile in range(len(listFiles_r)):
    fileString = listFiles_r[0].split('/')[-1].split('.')[-1]
    
    if iFile==0:
        uniqFile_r = fileString
        paths_r    = listFiles_r[iFile]
    else: 
        if fileString not in uniqFile_r:
            uniqFile_r = np.append(uniqFile, fileString)
            paths_r    = np.append(paths_r, listFiles_r[iFile])



In [30]:
print(len(paths_r),'\n',len(listFiles_r))

146 
 240


In [31]:
%%time 

## Open r files 
smyle_r = xr.open_mfdataset(paths_r, preprocess=preprocess_r_files, concat_dim='time', 
                           combine='nested', decode_times=True, 
                           chunks = {"time": -1, "lon": -1, "lat": -1},
                           data_vars='minimal', 
                         #  compat="override"
                         )


CPU times: user 240 ms, sys: 48.1 ms, total: 288 ms
Wall time: 322 ms


## Compare?

In [33]:
smyle_h0

<xarray.Dataset>
Dimensions:                          (levgrnd: 25, levlak: 10, levdcmp: 25,
                                      time: 240, hist_interval: 2, lon: 288,
                                      lat: 192, levsoi: 20, cft: 64,
                                      glc_nec: 10, ltype: 9, natpft: 15,
                                      nvegwcs: 4)
Coordinates:
  * levgrnd                          (levgrnd) float32 0.01 0.04 ... 28.87 42.0
  * levlak                           (levlak) float32 0.05 0.6 ... 34.33 44.78
  * levdcmp                          (levdcmp) float32 0.01 0.04 ... 28.87 42.0
  * time                             (time) object 1990-02-01 00:00:00 ... 20...
  * lon                              (lon) float32 0.0 1.25 2.5 ... 357.5 358.8
  * lat                              (lat) float32 -90.0 -89.06 ... 89.06 90.0
Dimensions without coordinates: hist_interval, levsoi, cft, glc_nec, ltype,
                                natpft, nvegwcs
Data variables: (12/481)
    mcdate                           (time) int32 dask.array<chunksize=(1,), meta=np.ndarray>
    mcsec                            (time) int32 dask.array<chunksize=(1,), meta=np.ndarray>
    mdcur                            (time) int32 dask.array<chunksize=(1,), meta=np.ndarray>
    mscur                            (time) int32 dask.array<chunksize=(1,), meta=np.ndarray>
    nstep                            (time) int32 dask.array<chunksize=(1,), meta=np.ndarray>
    time_bounds                      (time, hist_interval) object dask.array<chunksize=(1, 2), meta=np.ndarray>
    ...                               ...
    XSMRPOOL                         (time, lat, lon) float32 dask.array<chunksize=(1, 192, 288), meta=np.ndarray>
    XSMRPOOL_RECOVER                 (time, lat, lon) float32 dask.array<chunksize=(1, 192, 288), meta=np.ndarray>
    ZBOT                             (time, lat, lon) float32 dask.array<chunksize=(1, 192, 288), meta=np.ndarray>
    ZWT                              (time, lat, lon) float32 dask.array<chunksize=(1, 192, 288), meta=np.ndarray>
    ZWT_CH4_UNSAT                    (time, lat, lon) float32 dask.array<chunksize=(1, 192, 288), meta=np.ndarray>
    ZWT_PERCH                        (time, lat, lon) float32 dask.array<chunksize=(1, 192, 288), meta=np.ndarray>
Attributes: (12/102)
    title:                                     CLM History file information
    comment:                                   NOTE: None of the variables ar...
    Conventions:                               CF-1.0
    history:                                   created on 01/07/21 08:04:12
    source:                                    Community Land Model CLM4.0
    hostname:                                  cheyenne
    ...                                        ...
    cft_irrigated_tropical_corn:               62
    cft_tropical_soybean:                      63
    cft_irrigated_tropical_soybean:            64
    time_period_freq:                          month_1
    Time_constant_3Dvars_filename:             ./smyle_Transient.clm2.h0.1986...
    Time_constant_3Dvars:                      ZSOI:DZSOI:WATSAT:SUCSAT:BSW:H...

In [110]:
S2S_h0 = S2S_h0.indexes['time'].to_datetimeindex()


AttributeError: 'DatetimeIndex' object has no attribute 'to_datetimeindex'

In [99]:
s2s_monthly = S2S_h0.resample(time='1MS').mean(dim='time')

ValueError: index must be monotonic for resampling

In [102]:
for iTime in range(len(S2S_h0.time.values)-1):
    if S2S_h0.time.values[iTime+1]==S2S_h0.time.values[iTime]:
        print('Duplicate!')
        print('  Current iT: ', S2S_h0.time.values[iTime])
        print('  Next iT: ', S2S_h0.time.values[iTime+1])

    if S2S_h0.time.values[iTime] != S2S_h0.time.values[iTime]

<xarray.DataArray 'time' (time: 6794)>
array(['1989-12-22T00:00:00.000000000', '1989-12-23T00:00:00.000000000',
       '1989-12-24T00:00:00.000000000', ..., '2009-12-19T00:00:00.000000000',
       '2009-12-20T00:00:00.000000000', '2009-12-21T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 1989-12-22 1989-12-23 ... 2009-12-21

In [108]:
S2S_h0.time.values[0] + timedelta(days=1)

UFuncTypeError: ufunc 'add' cannot use operands with types dtype('<M8[ns]') and dtype('O')

In [109]:
S2S_h0.time.values[0]

numpy.datetime64('1989-12-22T00:00:00.000000000')